# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description. 

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |


In [1]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv("production_data.csv")

# Handle non-standard missing values
df.replace(['-', 'missing', 'Missing', 'MISSING', ''], np.nan, inplace=True)

# Remove rows with missing batch_id
df = df[df['batch_id'].notna()]

# Convert production_date to datetime
df['production_date'] = pd.to_datetime(df['production_date'], errors='coerce')

# raw_material_supplier: Fill missing with 1 first, then map
df['raw_material_supplier'] = df['raw_material_supplier'].fillna(1)
df['raw_material_supplier'] = df['raw_material_supplier'].map({1: 'national_supplier', 2: 'international_supplier'})

# Clean pigment_type (string): strip/standardize, then fill
df['pigment_type'] = df['pigment_type'].astype(str).str.strip().str.lower()
df['pigment_type'] = df['pigment_type'].fillna('other')
valid_pigments = ['type_a', 'type_b', 'type_c']
df['pigment_type'] = df['pigment_type'].where(df['pigment_type'].isin(valid_pigments), 'other')

# pigment_quantity: clean, clip to 1–100 range, fill missing with median
df['pigment_quantity'] = pd.to_numeric(df['pigment_quantity'], errors='coerce')
df.loc[(df['pigment_quantity'] < 1) | (df['pigment_quantity'] > 100), 'pigment_quantity'] = np.nan
df['pigment_quantity'] = df['pigment_quantity'].fillna(df['pigment_quantity'].median())

# mixing_time: numeric, fill with mean rounded to 2 decimals
df['mixing_time'] = pd.to_numeric(df['mixing_time'], errors='coerce')
df['mixing_time'] = df['mixing_time'].fillna(round(df['mixing_time'].mean(), 2))

# mixing_speed: clean up string casing, then fill missing
df['mixing_speed'] = df['mixing_speed'].astype(str).str.strip().str.title()
df['mixing_speed'] = df['mixing_speed'].replace('Nan', np.nan)
df['mixing_speed'] = df['mixing_speed'].fillna('Not Specified')

# product_quality_score: fill missing with mean rounded
df['product_quality_score'] = pd.to_numeric(df['product_quality_score'], errors='coerce')
df['product_quality_score'] = df['product_quality_score'].fillna(round(df['product_quality_score'].mean(), 2))

# Final cleaned DataFrame
clean_data = df


# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [2]:
import pandas as pd

# Load the data
df = pd.read_csv('production_data.csv')

# Clean the data as done in Task 1
df['production_date'] = pd.to_datetime(df['production_date'], errors='coerce')

df['raw_material_supplier'] = df['raw_material_supplier'].map({1: 'national_supplier', 2: 'international_supplier'})
df['raw_material_supplier'] = df['raw_material_supplier'].fillna('national_supplier')

df['pigment_type'] = df['pigment_type'].where(df['pigment_type'].isin(['type_a', 'type_b', 'type_c']), 'other')

df['pigment_quantity'] = pd.to_numeric(df['pigment_quantity'], errors='coerce')
df['pigment_quantity'] = df['pigment_quantity'].fillna(df['pigment_quantity'].median())

df['mixing_time'] = pd.to_numeric(df['mixing_time'], errors='coerce')
df['mixing_time'] = df['mixing_time'].fillna(round(df['mixing_time'].mean(), 2))

df['mixing_speed'] = df['mixing_speed'].fillna('Not Specified')

df['product_quality_score'] = pd.to_numeric(df['product_quality_score'], errors='coerce')
df['product_quality_score'] = df['product_quality_score'].fillna(round(df['product_quality_score'].mean(), 2))

#  Group and aggregate
aggregated_data = df.groupby('raw_material_supplier').agg(
    avg_product_quality_score=('product_quality_score', lambda x: round(x.mean(), 2)),
    avg_pigment_quantity=('pigment_quantity', lambda x: round(x.mean(), 2))
).reset_index()

# Display result
aggregated_data


,raw_material_supplier,avg_product_quality_score,avg_pigment_quantity
0,international_supplier,5.97,34.91
1,national_supplier,8.02,44.73


# Task 3

To get more insight into the factors behind product quality, you want to filter the data to see an average product quality score for a specified set of results.

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named pigment_data.
- It should consist of a 1-row DataFrame with 3 columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places where appropriate.


In [3]:
import pandas as pd

# Load the data
df = pd.read_csv('production_data.csv')

# Clean only relevant columns
df['raw_material_supplier'] = pd.to_numeric(df['raw_material_supplier'], errors='coerce')
df['pigment_quantity'] = pd.to_numeric(df['pigment_quantity'], errors='coerce')
df['product_quality_score'] = pd.to_numeric(df['product_quality_score'], errors='coerce')

# Filter for raw_material_supplier == 2 and pigment_quantity > 35
filtered = df[(df['raw_material_supplier'] == 2) & (df['pigment_quantity'] > 35)]

# Calculate averages
avg_pigment_quantity = round(filtered['pigment_quantity'].mean(), 2)
avg_quality_score = round(filtered['product_quality_score'].mean(), 2)

# Build the final 1-row DataFrame
pigment_data = pd.DataFrame({
    'raw_material_supplier': [2],
    'pigment_quantity': [avg_pigment_quantity],
    'avg_product_quality_score': [avg_quality_score]
})


# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [4]:
import pandas as pd
from scipy.stats import pearsonr

# Load the data
df = pd.read_csv('production_data.csv')

# Convert relevant columns to numeric
df['pigment_quantity'] = pd.to_numeric(df['pigment_quantity'], errors='coerce')
df['product_quality_score'] = pd.to_numeric(df['product_quality_score'], errors='coerce')

# Drop rows with missing values in the relevant columns
df = df.dropna(subset=['pigment_quantity', 'product_quality_score'])

# Calculate means and standard deviations
product_quality_score_mean = round(df['product_quality_score'].mean(), 2)
product_quality_score_sd = round(df['product_quality_score'].std(), 2)
pigment_quantity_mean = round(df['pigment_quantity'].mean(), 2)
pigment_quantity_sd = round(df['pigment_quantity'].std(), 2)

# Calculate Pearson correlation coefficient
corr_coef, _ = pearsonr(df['pigment_quantity'], df['product_quality_score'])
corr_coef = round(corr_coef, 2)

# Create the final output DataFrame
product_quality = pd.DataFrame([{
    'product_quality_score_mean': product_quality_score_mean,
    'product_quality_score_sd': product_quality_score_sd,
    'pigment_quantity_mean': pigment_quantity_mean,
    'pigment_quantity_sd': pigment_quantity_sd,
    'corr_coef': corr_coef
}])
